In [17]:
from datetime import datetime

import findspark
findspark.init()
import pymysql
import pandas.io.sql as psql
import pandas as pd
from pyspark.sql import SparkSession
import pyinputplus as pyip

ImportError: Unable to import required dependencies:
numpy: cannot import name randbits

In [25]:
from pyspark.sql import SparkSession
import findspark
findspark.init()


In [19]:
spark = SparkSession.builder.master("local[1]").appName('Credit_card.com').getOrCreate()

In [14]:
connection = pymysql.connect(
    host='localhost',
    user="root",
    password="password",
    database='creditcard_capstone'
)
cursor = connection.cursor()

In [ ]:
def transaction_menu_1_1(zipcode=21042, mm=11, yy=2018):
    sql = "SELECT c.CUST_ZIP,c.SSN,cc.CUST_SSN,cc.TRANSACTION_VALUE,cc.TRANSACTION_TYPE,cc.TIMEID FROM " \
          "cdw_sapp_customer c  JOIN cdw_sapp_credit_card cc ON c.SSN=cc.CUST_SSN"
    # cursor.execute(sql)
    # result_set = cursor.fetchall()
    data_frame = psql.read_sql(sql, con=connection)
    pd.set_option("display.max_columns", None)
    data_frame["TIMEID"] = pd.to_datetime(data_frame["TIMEID"], format='%Y%m%d')
    customer_transaction = data_frame[(data_frame.CUST_ZIP == zipcode) &
                                      (data_frame.TIMEID.dt.month == mm) &
                                      (data_frame.TIMEID.dt.year == yy)]
    daily_values = customer_transaction.groupby(customer_transaction.TIMEID.dt.day)["TRANSACTION_VALUE"].sum()
    result = pd.DataFrame({"Day": daily_values.index, "Transaction Value": daily_values.values})
    result = result.sort_values("Day", ascending=False)
print(result)

In [ ]:
def transaction_mod2(transaction_type='Bills'):
    sql = "SELECT c.CUST_ZIP,c.SSN,cc.CUST_SSN,cc.TRANSACTION_VALUE,cc.TRANSACTION_TYPE,cc.TIMEID \
           FROM cdw_sapp_customer c  JOIN cdw_sapp_credit_card cc ON c.SSN=cc.CUST_SSN "
    # cursor.execute(sql)
    data_frame = psql.read_sql(sql, con=connection)
    transaction_type = data_frame[
        (data_frame['TRANSACTION_TYPE'].str.lower() == transaction_type.lower()) & (data_frame['TRANSACTION_VALUE'])]
    transaction_df = transaction_type.groupby('TRANSACTION_TYPE').agg(count=('TRANSACTION_TYPE', 'count'),
                                                                      value=('TRANSACTION_VALUE', 'sum'))
    transaction_df['value'] = round(transaction_df['value'], 2)
    print(transaction_df)


In [ ]:
def transaction_mod3(branch_state='PA'):
    sql = "SELECT bc.BRANCH_CODE,bc.BRANCH_STATE,cc.TRANSACTION_VALUE,cc.TRANSACTION_TYPE,cc.BRANCH_CODE \
               FROM cdw_sapp_branch bc JOIN cdw_sapp_credit_card cc ON bc.BRANCH_CODE=cc.BRANCH_CODE"
    data_frame = psql.read_sql(sql, con=connection)
    branch_transaction = data_frame[(data_frame['BRANCH_STATE'] == branch_state) & (data_frame['TRANSACTION_VALUE'])]
    trans_by_branch_state = branch_transaction.groupby(['BRANCH_STATE']).agg(
        total_transactions=('TRANSACTION_VALUE', 'sum'))
    print(trans_by_branch_state)
